# Assignment 1 Deep Learning 
### Joshua Barry, S3718861

Data Loader

In [1]:
import pandas as pd
df = pd.read_csv("./data.csv")
print(df.head())
# Todo split the data reasonably 

                  View1                 View2                 View3  \
0  Image_000001_001.jpg  Image_000001_002.jpg  Image_000001_003.jpg   
1  Image_000002_001.jpg  Image_000002_002.jpg  Image_000002_003.jpg   
2  Image_000003_001.jpg  Image_000003_002.jpg  Image_000003_003.jpg   
3  Image_000004_001.jpg  Image_000004_002.jpg  Image_000004_003.jpg   
4  Image_000005_001.jpg  Image_000005_002.jpg  Image_000005_003.jpg   

                  View4     label  
0  Image_000001_004.jpg  airplane  
1  Image_000002_004.jpg  airplane  
2  Image_000003_004.jpg  airplane  
3  Image_000004_004.jpg  airplane  
4  Image_000005_004.jpg  airplane  


In [2]:
import tensorflow as tf 
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


### One Hot Encoding for the label

In [4]:
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv("./data.csv")
encoder=OneHotEncoder(sparse=False)
df_encoded = pd.DataFrame (encoder.fit_transform(df[['label']]))
df_encoded.columns = encoder.get_feature_names_out(['label'])
df.drop(['label'] ,axis=1, inplace=True)
df_encoded = pd.concat([df, df_encoded ], axis=1)
#print(df_encoded.head())

labels = ['airplane', 'bathtub', 'bench', 'bottle', 'bus', 'cabinet', 'car', 'chair',
                  'display', 'faucet', 'guitar', 'jar', 'lamp', 'laptop', 'loudspeaker', 'rifle', 
                  'sofa', 'table', 'telephone', 'watercraft']

def one_hot(row):
    list_ = [row['label_airplane'], row['label_bathtub'], row['label_bench'], row['label_bottle'], row['label_bus'], row['label_cabinet'], \
                row['label_car'], row['label_chair'], row['label_display'], row['label_faucet'], row['label_guitar'], row['label_jar'], \
                row['label_lamp'], row['label_laptop'], row['label_loudspeaker'], row['label_rifle'], row['label_sofa'], row['label_table'], \
                row['label_telephone'], row['label_watercraft']]
    row['one_hot'] = list_
    return row


df_encoded = df_encoded.apply(one_hot, axis=1)
print(df_encoded['one_hot'])
# df_encoded.drop['']

0        [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1        [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2        [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3        [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4        [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                               ...                        
30525    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
30526    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
30527    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
30528    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
30529    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
Name: one_hot, Length: 30530, dtype: object


Train Test Splitting

In [5]:
from sklearn.model_selection import train_test_split

with pd.option_context('mode.chained_assignment', None):
    train_data_df, test_data_df = train_test_split(df_encoded, test_size=0.2, shuffle=True, random_state=42)

print(train_data_df.shape[0], test_data_df.shape[0])

24424 6106


### Override methods

In [6]:
import tensorflow.keras as keras
import tensorflow as tf
from scipy.interpolate import interp1d
import numpy as np
from scipy.ndimage.interpolation import rotate, shift
from PIL import Image

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data_frame, batch_size=8, dim=(64, 64, 3), n_classes=20, data_mean=0, data_std=1,  data_prefix='./Data/Train/', shuffle=True, Augment=True):
        'Initialization'
        self.dim = dim  # Dimentions of the input
        self.batch_size = batch_size
        self.n_classes = n_classes  
        self.shuffle = shuffle  
        self.Augment = Augment  
        self.data_frame = data_frame
        self.indexes = range(len(df_encoded))
        self.image_label = data_frame['one_hot'].values.tolist()
        self.image_ids = data_frame[['View1', 'View2', 'View3', 'View4']].values.tolist()
        # needs to become a list of list, each list has 4 views for each row
        self.data_prefix = data_prefix
        
        # Data normalization parameters
        self.data_mean = data_mean
        self.data_std = data_std
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.image_ids) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data for the given index'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        data_ids_temp = [self.image_ids[k] for k in indexes]
        image_label_temp = [self.image_label[k] for k in indexes]
        
        # Generate data
        X, y = self.__data_generation(data_ids_temp, image_label_temp)
        
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.image_ids))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    # Support function
    def __data_generation(self, data_ids_temp, image_label_temp):
        'Generates data containing batch_size samples' 
        labels = []
        view1 = []
        view2 = []
        view3 = []
        view4 = []    

        # Generate data
        for i, ids in enumerate(data_ids_temp):
            view1.append(self.__read_data_instance(data_ids_temp[i][0]))
            view2.append(self.__read_data_instance(data_ids_temp[i][1]))
            view3.append(self.__read_data_instance(data_ids_temp[i][2]))
            view4.append(self.__read_data_instance(data_ids_temp[i][3]))
            labels.append(image_label_temp[i])
        views = [view1, view2, view3, view4]
        output = [tf.convert_to_tensor(i) for i in views]
        return output, np.asarray(labels)


    def __read_data_instance(self, label):
      # Read an image
      filepath = self.data_prefix + label
      
      data = Image.open(filepath)
      #data = data.resize((224,32))
      data = np.asarray(data)
      #data = np.expand_dims(data, axis=0) 

      if self.Augment:
          rot = np.random.rand(1) < 0.5
          if rot:
              rot = np.random.randint(-10,10, size=1)
              data = rotate(data, angle=rot[0], reshape=False)
          
          #shift_val = np.random.randint(-5, high=5, size=2, dtype=int).tolist() + [0,]
          #data = shift(data, shift_val, order=0, mode='constant', cval=0.0, prefilter=False)

      X = data

      # Input normalization
      X = (X - self.data_mean)/self.data_std
      return X

datagen = DataGenerator(df_encoded, batch_size=8, data_mean=0, data_std=1, data_prefix='./Train/', shuffle=True, Augment=True)
#print(datagen.__getitem__(0)[0])

In [12]:
from tensorflow.keras import Model
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Concatenate, BatchNormalization, Activation, GlobalAveragePooling2D, Wrapper, Maximum
from tensorflow.keras.optimizers import RMSprop, Adam

resnet = ResNet50V2(input_shape=(64,64,3), include_top=False, weights='imagenet')

for i in resnet.layers:
    i.trainable = False

class Wrapper(Model):

    def __init__(self, model):
        super(Wrapper, self).__init__()
        self.image1 = Input(shape=(64,64,3), batch_size=8)
        self.image2 = Input(shape=(64,64,3), batch_size=8)
        self.image3 = Input(shape=(64,64,3), batch_size=8)
        self.image4 = Input(shape=(64,64,3), batch_size=8)
        # need to to inout now 

        self.resnet = model
        self.max_layer = Maximum()
        self.GAB = GlobalAveragePooling2D()
        self.dense1 = Dense(720, activation='relu')
        self.drop1 = tf.keras.layers.Dropout(0.2)
        self.dense2 = Dense(512, activation='relu')
        self.drop2 = tf.keras.layers.Dropout(0.2)
        self.dense3 = Dense(20, activation='softmax')
    
    def call(self, input):
        in1 = input[0]
        in2 = input[1]
        in3 = input[2]
        in4 = input[3]
        x1 = self.resnet(in1)
        x2 = self.resnet(in2)
        x3 = self.resnet(in3)
        x4 = self.resnet(in4)
        x1 = self.GAB(x1)
        x2 = self.GAB(x2)
        x3 = self.GAB(x3)
        x4 = self.GAB(x4)
        x = self.max_layer([x1, x2, x3, x4])
        x = self.dense1(x)
        x = self.drop1(x)
        x = self.dense2(x)
        x = self.drop2(x)
        x = self.dense3(x)
        return x

model = Wrapper(resnet)
model.compile(optimizer=RMSprop(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
tensorboard_callback = TensorBoard(log_dir="./tensorboard/experiment2/")
checkpoint_callback = ModelCheckpoint(
    filepath='./checkpoints/experiment2/',
    save_weights_only=True,
    monitor='accuracy',
    mode='max')
early_stop = EarlyStopping(monitor='loss', patience=4, restore_best_weights=True)
model.fit(datagen, epochs=10, callbacks=[tensorboard_callback, checkpoint_callback, early_stop])

Epoch 1/10
2799/3816 [=====================>........] - ETA: 4:53 - loss: 11.3508 - accuracy: 0.1601

In [ ]:
# Batch size of 32 is good
from turtle import shape
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation
from keras.models import Sequential
# tf.keras.Input
from keras import Input as input

def createModel():
    # 4 inputs, 4 images
    input1 = input(shape=(64, 64, 3), batch_size = 8)
    input2 = input(shape=(64, 64, 3), batch_size = 8)
    input3 = input(shape=(64, 64, 3), batch_size = 8)
    input4 = input(shape=(64, 64, 3), batch_size = 8)

    convA = Conv2D(32, (3,3), padding="same")(input1)
    convA = MaxPooling2D(pool_size=(2, 2))(convA)
    convA = Activation("relu")(convA)
    # To prevent overfitting
    convA = Dropout(0.25)(convA)

    convA = Conv2D(64, (3,3), padding="same")(convA)
    convA = MaxPooling2D(pool_size=(2, 2))(convA)
    convA = Activation("relu")(convA)

    convB = Conv2D(32, (3,3), padding="same")(input2)
    convB = MaxPooling2D(pool_size=(2, 2))(convB)
    convB = Activation("relu")(convB)
    convB = Dropout(0.25)(convB)

    convB = Conv2D(64, (3,3), padding="same")(convB)
    convB = MaxPooling2D(pool_size=(2, 2))(convB)
    convB = Activation("relu")(convB)

    convC = Conv2D(32, (3,3), padding="same")(input3)
    convC = MaxPooling2D(pool_size=(2, 2))(convC)
    convC = Activation("relu")(convC)
    convC = Dropout(0.25)(convC)

    convC = Conv2D(64, (3,3), padding="same")(convC)
    convC = MaxPooling2D(pool_size=(2, 2))(convC)
    convC = Activation("relu")(convC)

    convD = Conv2D(32, (3,3), padding="same")(input4)
    convD = MaxPooling2D(pool_size=(2, 2))(convD)
    convD = Activation("relu")(convD)
    convD = Dropout(0.25)(convD)

    convD = Conv2D(64, (3,3), padding="same")(convD)
    convD = MaxPooling2D(pool_size=(2, 2))(convD)
    convD = Activation("relu")(convD)

    # Flatten the data for the dense layers
    flatA = Flatten()(convA)
    flatB = Flatten()(convB)
    flatC = Flatten()(convC)
    flatD = Flatten()(convD)

    # merge them together
    # 4 1d vects
    # 1 1d vector preserving as much info as i can 
    print(flatA)
    print(flatA.shape)

    # Concatenate the data
    merged = tf.math.reduce_max([flatA, flatB, flatC, flatD], axis=0)
    print(merged)
    print(merged.shape)

    # Dense layers
    dense = Dense(512)(merged)
    dense = Activation("relu")(dense)
    dense = Dropout(0.5)(dense)

    # Output layer
    output = Dense(1, activation="sigmoid")(dense)
    
    model = tf.keras.Model(inputs=[input1, input2, input3, input4], outputs=output)
    return model


def summary(model):
    model.summary()
    for layer in model.layers:
        print(layer.output_shape)  


# Compile the model

model = createModel()
model.summary()

# print(model.layers[0])

In [ ]:
# Compile the model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

# Train the model


model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

# Train the model
model.fit(datagen, epochs=10)

In [ ]:
# Global average pooling 
# flatten
# hyper parameter tuning

In [ ]:
training_generator = DataGenerator(df_encoded, batch_size=32, data_mean=0, data_std=1, data_prefix='./Data/Train/', shuffle=True, Augment=True)
validation_generator = DataGenerator(df_encoded, batch_size=32, data_mean=0, data_std=1, data_prefix='./Data/Train/', shuffle=True, Augment=True)

model = createModel()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Save the model
model.save('model.h5')


In [ ]:
model.summary() # Print the model summary

# Writing Custom CNN Layer

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):

    # Initialize components of the model
    def __init__(self, filter_num, stride=1, reg_lambda=0.0):
        super(ResidualBlock, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            kernel_initializer="he_normal",
                                            kernel_regularizer=tf.keras.regularizers.l2(reg_lambda),
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization(momentum=.4)
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            kernel_initializer="he_normal",
                                            kernel_regularizer=tf.keras.regularizers.l2(reg_lambda),
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization(momentum=.4)
        if stride != 1:
            self.downsample = tf.keras.Sequential()
            self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num,
                                                       kernel_size=(1, 1),
                                                       kernel_initializer="he_normal",
                                                       kernel_regularizer=tf.keras.regularizers.l2(reg_lambda),
                                                       strides=stride))
            self.downsample.add(tf.keras.layers.BatchNormalization(momentum=.4))
        else:
            self.downsample = lambda x: x

    # Define the forward function
    def call(self, inputs, training=None, **kwargs):
        residual = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)

        output = tf.nn.relu(tf.keras.layers.add([residual, x]))

        return output

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'conv1': self.conv1,
            'bn1': self.bn1,
            'conv2': self.conv2,
            'bn2': self.bn2,
            'downsample': self.downsample,
        })
        return config

In [ ]:
def get_resnet_model(filters, block_size, reg_lambda=0.0, fdropout=False):
  model = tf.keras.Sequential()

  #initial segment
  model.add(tf.keras.layers.Conv2D(filters=64,
                                   kernel_size=(3, 3),
                                   strides=1,
                                   kernel_initializer="he_normal",
                                   kernel_regularizer=tf.keras.regularizers.l2(reg_lambda),
                                   padding="same", input_shape=(32, 32, 3)))
  model.add(tf.keras.layers.BatchNormalization(momentum=.4))

  #Stack of residual blocks
  for nFilters, nBlocks in zip(filters, block_size):
    model.add(ResidualBlock(nFilters, stride=2, reg_lambda=reg_lambda))
    
    for _ in range(1, nBlocks):
      model.add(ResidualBlock(nFilters, stride=1, reg_lambda=reg_lambda))

  # Final part
  model.add(tf.keras.layers.GlobalAveragePooling2D())
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(10, 
                                  activation=tf.nn.softmax, 
                                  kernel_regularizer=tf.keras.regularizers.l2(reg_lambda),
                                  kernel_initializer="he_normal"))

  return model

In [ ]:
# # Load the model
# from keras.models import load_model
# model = load_model('model.h5')

# # Evaluate the model
# score = model.evaluate(, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

# # Predict the model
# predictions = model.predict_generator(generator=validation_generator)
# print(predictions)

# # Save the predictions
# np.save('predictions.npy', predictions)

# # Load the predictions
# predictions = np.load('predictions.npy')

# # Plot the predictions
# import matplotlib.pyplot as plt
# plt.plot(predictions)
# plt.show()

# TODO: Create RESNOV Model

TODO: Data Augmentation 